In [25]:
import pandas as pd
%load_ext kedro.ipython

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


In [26]:
df= catalog.load("pima_imputed")
df.head()

[06/01/25 13:49:58] INFO     Loading data from pima_imputed (CSVDataset)...                     ]8;id=356226;file:///Users/rkw/miniconda3/envs/kedro312/lib/python3.12/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=176372;file:///Users/rkw/miniconda3/envs/kedro312/lib/python3.12/site-packages/kedro/io/data_catalog.py#403\403]8;;\

,pregnant,glucose,blood_pr,skin_thi,insulin,bmi,dbts_pdgr,age,flag_imp,outcome
0,6.0,148.0,72.0,35.000000,127.477659,33.6,0.627000,50.0,True,1.0
1,1.0,85.0,66.0,29.000000,120.094622,26.6,0.351000,31.0,True,0.0
2,8.0,183.0,64.0,16.168751,298.445158,23.3,0.672000,32.0,True,1.0
3,1.0,89.0,66.0,23.000000,94.000000,28.1,0.167000,21.0,False,0.0
4,0.0,137.0,40.0,35.000000,168.000000,43.1,0.739128,33.0,True,1.0


In [27]:
# 1) Identify feature columns (exclude outcome and flag)
df_subset = df.drop(columns=["flag_imp"])
df_subset

,pregnant,glucose,blood_pr,skin_thi,insulin,bmi,dbts_pdgr,age,outcome
0,6.0,148.0,72.0,35.000000,127.477659,33.6,0.627000,50.0,1.0
1,1.0,85.0,66.0,29.000000,120.094622,26.6,0.351000,31.0,0.0
2,8.0,183.0,64.0,16.168751,298.445158,23.3,0.672000,32.0,1.0
3,1.0,89.0,66.0,23.000000,94.000000,28.1,0.167000,21.0,0.0
4,0.0,137.0,40.0,35.000000,168.000000,43.1,0.739128,33.0,1.0
...,...,...,...,...,...,...,...,...,...
763,10.0,101.0,76.0,48.000000,180.000000,32.9,0.171000,63.0,0.0
764,2.0,122.0,70.0,27.000000,92.491162,36.8,0.340000,27.0,0.0
765,5.0,121.0,72.0,23.000000,112.000000,26.2,0.245000,30.0,0.0
766,1.0,126.0,60.0,22.224915,232.324745,30.1,0.349000,47.0,1.0


In [29]:
from scipy.stats import ttest_ind
# 1) Identify feature columns (everything except "outcome")
features = [col for col in df_subset.columns if col != "outcome"]

# 2) Loop through each feature, split by outcome, and run a t-test
results = []
for feat in features:
    grp0 = df_subset.loc[df_subset["outcome"] == 0, feat].dropna().astype(float)
    grp1 = df_subset.loc[df_subset["outcome"] == 1, feat].dropna().astype(float)

    tstat, pval = ttest_ind(grp0, grp1, equal_var=False)
    results.append({
        "feature": feat,
        "t_stat": tstat,
        "p_value": pval
    })

# 3) Convert to DataFrame for easy viewing
t_test_results = pd.DataFrame(results)
print(t_test_results)

     feature     t_stat       p_value
0   pregnant  -5.652359  2.724766e-08
1    glucose -14.913068  1.812152e-41
2   blood_pr  -4.855722  1.566151e-06
3   skin_thi  -6.919344  1.204014e-11
4    insulin  -8.386379  6.071688e-16
5        bmi  -9.012518  2.896490e-18
6  dbts_pdgr  -5.126683  4.365635e-07
7        age  -7.329143  8.134674e-13


Signature:
ttest_ind(
    a,
    b,
    *,
    axis=0,
    equal_var=True,
    nan_policy='propagate',
    permutations=None,
    random_state=None,
    alternative='two-sided',
    trim=0,
    method=None,
    keepdims=False,
)
Docstring:
Calculate the T-test for the means of *two independent* samples of scores.

This is a test for the null hypothesis that 2 independent samples
have identical average (expected) values. This test assumes that the
populations have identical variances by default.

.. deprecated:: 1.17.0
    Use of argument(s) ``{'trim', 'nan_policy', 'equal_var', 'keepdims', 'axis', 'method', 'alternative'}`` by position is deprecated; beginning in 
    SciPy 1.17.0, these will be keyword-only. Argument(s) ``{'permutations', 'random_state'}`` are deprecated, whether passed by position or keyword; they will be removed in SciPy 1.17.0. Use ``method`` to perform a permutation test.

Parameters
----------
a, b : array_like
    The arrays must have the same shape, except in t